## Part 3 - Monitoring roads

#### Imports

In [1]:
import pandas as pd
import numpy as np
import folium
import sys
from IPython.display import display
import ipywidgets as widgets

### Load data

#### Accidents data

In [2]:
def load_dataset():  
    veh_cols = ['Num_Acc', 'num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc']
    usa_cols = ['Num_Acc', 'num_veh', 'place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu', 'locp']
    lie_cols = ['Num_Acc', 'catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ']
    car_cols = ['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'atm', 'col', 'gps', 'lat', 'long']

    veh = pd.read_csv('../Accidents/vehicules_2005.csv', usecols=veh_cols)
    usa = pd.read_csv('../Accidents/usagers_2005.csv', usecols=usa_cols)
    lie = pd.read_csv('../Accidents/lieux_2005.csv', usecols=lie_cols)
    car = pd.read_csv('../Accidents/caracteristiques_2005.csv', encoding='latin-1', usecols=car_cols)

    for i in range(2006, 2018, 1):
        lie = pd.concat([lie, pd.read_csv('../Accidents/lieux_' + str(i) + '.csv', usecols=lie_cols)], axis=0)
        usa = pd.concat([usa, pd.read_csv('../Accidents/usagers_' + str(i) + '.csv', usecols=usa_cols)], axis=0)
        veh = pd.concat([veh, pd.read_csv('../Accidents/vehicules_' + str(i) + '.csv', usecols=veh_cols)], axis=0)
        try:
            car = pd.concat([car, pd.read_csv('../Accidents/caracteristiques_' + str(i) + '.csv',
                                              encoding='latin-1', usecols=car_cols)], axis=0)
        except:
            car = pd.concat(
                [car, pd.read_csv('../Accidents/caracteristiques_' + str(i) + '.csv', usecols=car_cols,
                                  encoding='latin-1', sep='\t')], axis=0)
            
    return veh, usa, car, lie

veh, usa, car, lie = load_dataset()

car.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,atm,col,com,gps,lat,long,dep
0,200500000001,5,1,12,1900,3,2,1.0,3.0,11.0,M,5051500.0,294400,590
1,200500000002,5,1,21,1600,1,2,1.0,1.0,51.0,M,5053700.0,280200,590
2,200500000003,5,1,21,1845,3,1,2.0,1.0,51.0,M,5054600.0,280000,590
3,200500000004,5,1,4,1615,1,1,1.0,5.0,82.0,M,5098700.0,240800,590
4,200500000005,5,1,10,1945,3,1,3.0,6.0,478.0,M,5096400.0,247500,590


#### Correspondance data

With this table, we have a correspondance between communal code in our accidents dataset, and the postal code, which is more convenient to identify a city.

In [33]:
com_postal_pivot.iloc[:20000, :].to_csv('postal_part1.csv')
com_postal_pivot.iloc[20000:, :].to_csv('postal_part2.csv')

In [42]:
postal_1 = pd.read_csv('../Accidents/postal_part1.csv')
postal_2 = pd.read_csv('../Accidents/postal_part2.csv')

com_postal_pivot = pd.concat([postal_1, postal_2], ignore_index=True).set_index('Code_postal')
com_postal_pivot.head(5)

,Geo Point,CODE_COM,INSEE_COM,NOM_COM,SUPERFICIE,POPULATION,CODE_DEPT,NOM_DEPT,CODE_REG,NOM_REG,Libelle_acheminement,Nom_commune
Code_postal,,,,,,,,,,,,
95450.0,"49.0848358031,1.9748748362",2,95002,ABLEIGES,792.0,933,95,VAL-D'OISE,11,ILE-DE-FRANCE,ABLEIGES,ABLEIGES
76280.0,"49.6097472297,0.241445095242",716,76716,TURRETOT,612.0,1511,76,SEINE-MARITIME,28,NORMANDIE,TURRETOT,TURRETOT
76690.0,"49.6093542995,1.07016467145",105,76105,LE BOCASSE,860.0,707,76,SEINE-MARITIME,28,NORMANDIE,LE BOCASSE,LE BOCASSE
10290.0,"48.3605004241,3.59036233813",54,10054,BOURDENAY,1856.0,128,10,AUBE,44,ALSACE-CHAMPAGNE-ARDENNE-LORRAINE,BOURDENAY,BOURDENAY
51220.0,"49.345648783,3.97511257054",329,51329,LOIVRE,1024.0,1197,51,MARNE,44,ALSACE-CHAMPAGNE-ARDENNE-LORRAINE,LOIVRE,LOIVRE


## Filtering on postal code

In [43]:
def postal_code_selection(code = None):
    
    # Handle given postal code (optional)
    if isinstance(code, str): 
        postal_code_input = code
    else: 
        postal_code_input = input("Select a postal code : ")
        
    try:
        result = com_postal_pivot.loc[int(postal_code_input) , : ]
    except:
        print('Postal code not found in database')

    if isinstance(result, pd.Series):
        #print("Single line result")
        return result
    elif isinstance(result, pd.DataFrame):
        #print("Multi line result") Get first row
        return result.iloc[0]
    else:
        print(result.shape)

postal_code_selection('75015')

Geo Point               48.840040811,2.29268840406
CODE_COM                                       115
INSEE_COM                                    75115
NOM_COM                   PARIS-15E-ARRONDISSEMENT
SUPERFICIE                                     846
POPULATION                                  238190
CODE_DEPT                                       75
NOM_DEPT                                     PARIS
CODE_REG                                        11
NOM_REG                              ILE-DE-FRANCE
Libelle_acheminement                         PARIS
Nom_commune                               PARIS 15
Name: 75015.0, dtype: object

## Map drawing

In [8]:
def draw_map(
    scale='City',
    post_code='75013', 
    department='75',
    veh_type='all', 
    weather='all', 
    luminosity='all', 
    gravity='all',
    usa_sexe='all',
    export_name='plot_map.html',
    begin='16', end='18'):

    # Cross data - Filter on Dpt & Com
    if scale == 'City':
        result = postal_code_selection(post_code)
        display_scale = 14
        car_tmp = car[(car['dep'] == int(result['CODE_DEPT']+'0')) & (car['com'] == result['CODE_COM'])]
        centered_location = [float(result["Geo Point"].split(",")[0]), float(result["Geo Point"].split(",")[1])]
    elif scale == 'Department':
        car_tmp = car[(car['dep'] == int(str(department) + '0'))]
        if department == '75':
            display_scale = 12
            centered_location = [48.866997, 2.339459]
        else:
            display_scale = 9
            result = postal_code_selection(department+'000')
            centered_location = [float(result["Geo Point"].split(",")[0]), float(result["Geo Point"].split(",")[1])]
    elif scale == 'Country':
        car_tmp = car
        display_scale = 6
        centered_location = [47.12, 2.5]
    
    zone_df = pd.merge(veh, car_tmp, on='Num_Acc')
    zone_df = pd.merge(zone_df, lie, on='Num_Acc')
    zone_df = zone_df[zone_df.gps == 'M']
    
    # Filter on road type
    if scale == 'Country':
        zone_df = zone_df[zone_df.catr == 1.0]

    # Filter empty coordinates
    zone_df = zone_df[(zone_df.long != 0) & (zone_df.lat != 0)]
    zone_df = zone_df[(zone_df.long != 0.0) & (zone_df.lat != 0.0)]
    zone_df = zone_df.dropna(subset=['lat', 'long'])
    
    # Filter on vehicle type
    if (veh_type == 'Cars'):
        zone_df = zone_df[(zone_df.catv == 7)]
    elif veh_type == 'Motorcycles':
        zone_df = zone_df[(zone_df.catv == 2) | (zone_df.catv == 30) | (zone_df.catv == 31) | 
                          (zone_df.catv == 32) | (zone_df.catv == 33) | (zone_df.catv == 34)]
    elif veh_type == 'Bikes':
        zone_df = zone_df[(zone_df.catv == 1)]
    elif veh_type == 'Public transports':
        zone_df = zone_df[(zone_df.catv == 37) | (zone_df.catv == 38) | (zone_df.catv == 39) | 
                          (zone_df.catv == 40)]
        
    # Luminosity conditions
    if (luminosity == 'Day'):                   zone_df = zone_df[(zone_df.lum == 1)]
    elif luminosity == 'Dusk/Dawn':             zone_df = zone_df[(zone_df.lum == 2)]
    elif luminosity == 'Night (with lighting)': zone_df = zone_df[(zone_df.lum == 5)]
    elif luminosity == 'Night (no lighting)':   zone_df = zone_df[(zone_df.lum == 3) | (zone_df.lum == 4)]
    
    # Atmospheric conditions
    if weather == 'Normal/Sunny':   zone_df = zone_df[(zone_df.atm == 1) | (zone_df.atm == 7)]
    elif weather == 'Rain':         zone_df = zone_df[(zone_df.atm == 2) | (zone_df.atm == 3)]
    elif weather == 'Snow':         zone_df = zone_df[(zone_df.atm == 4)]
    elif weather == 'Fog':          zone_df = zone_df[(zone_df.atm == 5)]
        
    # Filter on time interval
    zone_df = zone_df[(zone_df.an >= int(begin)) & (zone_df.an <= int(end))]
    
    # Filter on accident gravity
    if gravity == 'Hurt':
        zone_df = pd.merge(zone_df, usa, on='Num_Acc')
        zone_df = zone_df[(zone_df.grav == 3) | (zone_df.grav == 4)]
    elif gravity == 'Killed':
        zone_df = pd.merge(zone_df, usa, on='Num_Acc')
        zone_df = zone_df[(zone_df.grav == 2)]
    elif gravity == 'Hurt or killed':
        zone_df = pd.merge(zone_df, usa, on='Num_Acc')
        zone_df = zone_df[(zone_df.grav == 2) | (zone_df.grav == 3) | (zone_df.grav == 4)]
        
    # Filter on persons sex
    if 'sexe' in zone_df.columns:
        if usa_sexe == 'male':
            zone_df = zone_df[(zone_df.sexe == 1)]
        elif usa_sexe == 'female':
            zone_df = zone_df[(zone_df.sexe == 2)]
    elif (usa_sexe != 'all') and ('sexe' not in zone_df.columns):
        zone_df = pd.merge(zone_df, usa, on='Num_Acc')
        if usa_sexe == 'male':
            zone_df = zone_df[(zone_df.sexe == 1)]
        elif usa_sexe == 'female':
            zone_df = zone_df[(zone_df.sexe == 2)]
    
    # Coordinates conversion
    zone_df['lat_folium'] = zone_df['lat'].replace('-', 0).fillna(0).astype('float') / 100000
    zone_df['long_folium'] = zone_df['long'].replace('-', 0).fillna(0).astype('float') / 100000
    zone_df = zone_df[zone_df.long_folium != 0.0]
    zone_df = zone_df.drop(['lat', 'long'], axis=1)
    #print(zone_df.shape[0], ' rows with good gps cordinates.\n')
    
    map_osm = folium.Map(location=centered_location,
                         zoom_start=display_scale)

    for ind, lat, lon in zone_df[['lat_folium', 'long_folium']].itertuples():

        map_osm.add_child(folium.RegularPolygonMarker(
            location=[lat,lon],
            fill_color='#0f51dd',
            radius=5,
            popup='Vehicle code : '+str(zone_df.loc[ind, 'catv'])+\
            '<br> Year : 20'+str(zone_df.loc[ind, 'an'])+\
            '<br> Road Type : '+str(zone_df.loc[ind, 'catr'])))
    
    map_osm.save(export_name)
    display(map_osm)
    
#draw_map('33000', veh_type='all')

In [16]:
graph_scale = widgets.ToggleButtons(
    options=['City', 'Department', 'Country'],
    description='Scale :')

vehicle_types = widgets.ToggleButtons(
    options=['All', 'Cars', 'Motorcycles', 'Bikes', 'Public transports'],
    description='Vehicle type:')

luminosity = widgets.ToggleButtons(
    options=['All', 'Day', 'Dusk/Dawn', 'Night (with lighting)', 'Night (no lighting)'],
    description='Luminosity :')

weather = widgets.ToggleButtons(
    options=['All', 'Normal/Sunny', 'Rain', 'Snow', 'Fog'], 
    description='Weather:')

postal_code = widgets.Text(
    value='75015', placeholder='Type 5-digits value',
    description='Post. Code:', disabled=False)

department = widgets.Text(
    value='75', placeholder='Type 2-digits value',
    description='Department:', disabled=False)

usa_sexe = widgets.ToggleButtons(
    options=['all', 'male', 'female'],
    description='Sex :')

gravity = widgets.ToggleButtons(
    options=['all', 'Hurt', 'Killed', 'Hurt or killed'],
    description='Gravity :')

time_window = widgets.IntRangeSlider(
    value=[2016, 2018], min=2006, max=2018,
    step=1, description='Year :',
    disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='d'
)


def event_handler(btn_object):
    
    draw_map(scale = graph_scale.value,
             post_code = postal_code.value,
             department = department.value,
             veh_type = vehicle_types.value, 
             begin = str(time_window.value[0])[2:4], 
             end = str(time_window.value[1])[2:4],
             weather = weather.value, 
             luminosity = luminosity.value,
             gravity = gravity.value,
             usa_sexe = usa_sexe.value,
             export_name = 'tls_killed.html')
    

go_btn = widgets.Button(description="Display map")
go_btn.on_click(event_handler)

display(graph_scale, vehicle_types, weather, luminosity, postal_code, department, time_window, 
        usa_sexe, gravity, go_btn)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget